In [2]:
# Import the libraries
import pandas as pd
import numpy as np
import requests
import datetime
import statistics


<ipython-input-2-2455503a6f1f>:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# Ingest the wheat yield data and organize it
years = pd.read_csv('wheat_yield.csv')
helper = {}
for i in years.columns:
    helper[i]= years[i][2]
df_wheat_yield = pd.DataFrame(list(helper.items()), columns=['Years', 'Yield'])
df_wheat_yield

,Years,Yield
0,1950,25.7
1,1951,28.6
2,1952,27.5
3,1953,27.4
4,1954,26.1
...,...,...
69,2019,74.0
70,2020,78.2
71,2021,73.0
72,2022,75.8


## Drought data for the wheat growth cycle

## Transform API weather data

In [4]:
# Ingest the weather data
df_weather = pd.read_csv("daily_datax.csv")             
df_weather.head()

,Column1,date,temperature_2m_max,temperature_2m_min,sunshine_duration,precipitation_sum,et0_fao_evapotranspiration
0,0,1/1/1950 2:00,-0.163750,-4.355417,19761.6620,0.083333,14.410975
1,1,1/2/1950 2:00,4.544583,-2.747083,7741.8267,4.333334,24.274849
2,2,3/1/1950 2:00,0.236250,-7.705417,25016.4670,0.216667,10.503711
3,3,3/2/1950 2:00,1.211250,-7.180417,30359.3670,0.083333,11.052758
4,4,3/3/1950 2:00,3.602917,-6.455417,32220.7340,0.300000,12.135287


In [5]:
# Remove duplicate index column
df_weather = df_weather.drop(df_weather.columns[0], axis=1)

# Transform datetime data to pd standard
df_weather['date'] = pd.to_datetime(df_weather['date'])

df_weather.columns   

Index(['date', 'temperature_2m_max', 'temperature_2m_min', 'sunshine_duration',
       'precipitation_sum', 'et0_fao_evapotranspiration'],
      dtype='object')

In [6]:
# Filter between the relevant dates
df_weather = df_weather[df_weather['date'].dt.strftime('%m-%d').between('03-01','08-10')]

# Calculate Growing Degree Days (GDD)
df_weather['temperature_2m_avg']= (df_weather['temperature_2m_max'] + df_weather['temperature_2m_min'])/2

# Replace avg <0 with 0 
df_weather.loc[df_weather['temperature_2m_avg'] < 0, 'temperature_2m_avg'] = 0

# Converting seconds to hours 
df_weather['sunshine_duration'] = df_weather['sunshine_duration'].div(3600).round(2)
df_weather['sunshine_duration']

2         6.95
3         8.43
4         8.95
5         3.87
6         3.54
         ...  
13778     5.01
13779     8.94
13780    11.42
13781     8.07
13782    13.00
Name: sunshine_duration, Length: 12225, dtype: float64

In [7]:
# Create dataframe with yearly data
years_df = pd.DataFrame({
    'Year': df_weather['date'].dt.year.unique()
})

# Create dictionary for storing future values in years_df
cols_dict = {}

# Iterations to create yearly data on new df
for index in range(1950, 2023+1):
    # Filter df for the current year
    filtered_data = df_weather[df_weather['date'].dt.year == index]

    # Count days when Temperature > 30
    days_above_30 = filtered_data[filtered_data['temperature_2m_max'] > 30].shape[0]


    # Precipitation avg for the 6-month period
    precipitation_sum = filtered_data['precipitation_sum'].sum()

    # Sunshine hours total and averages
    sunshine_avg = filtered_data['sunshine_duration'].mean()

    # Growing Degree Days (GDD)
    gdd = filtered_data['temperature_2m_avg'].sum()

    # Evapotranspiration total
    et0_fao_evapotranspiration = filtered_data['et0_fao_evapotranspiration'].sum()
    
    # Store count in the dictionary
    cols_dict[index] = {
        'days_above_30degrees' : days_above_30,
        'precipitation_sum' : precipitation_sum,
        'sunshine_avg' : sunshine_avg,
        'growing_degree_days' : gdd,
        'evapotranspiration_sum' : et0_fao_evapotranspiration 
        
    }

In [8]:
# Assign the values from the dictionary to new columns in years_df
for name in cols_dict[years_df['Year'].iloc[0]].keys():  # Use the first year to get the keys
    years_df[name] = years_df['Year'].map(lambda x: cols_dict[x][name])

years_df

,Year,days_above_30degrees,precipitation_sum,sunshine_avg,growing_degree_days,evapotranspiration_sum
0,1950,0,326.250000,10.092393,2098.062491,2624.882042
1,1951,0,371.450004,9.824110,1904.548318,2706.235736
2,1952,3,272.266668,10.015828,2083.840007,2531.561542
3,1953,0,354.533338,10.326994,2070.079572,2575.040627
4,1954,0,380.433329,9.388405,1849.079572,2692.333876
...,...,...,...,...,...,...
69,2019,8,357.533335,9.877975,2326.124013,3028.859173
70,2020,6,315.533337,10.407362,2231.015675,2884.648170
71,2021,3,467.833338,9.969018,2062.932352,2851.701592
72,2022,8,262.900004,10.853436,2295.799016,2829.613592


### Adding SPI (Standard Precipitation Index) in DataFrame
SPI = (P-P*) /  σp
P = precipitation
p* = mean precipitation
σp = standard deviation of precipitation

In [9]:
# SPI Drought Index : values & mean
values = [year['precipitation_sum'] for year in cols_dict.values()]
mean_value = np.mean(values)
# Standard deviation : 
std = statistics.stdev(values)
# Create new column SPI
years_df['SPI'] = (years_df['precipitation_sum'] - mean_value)/std

In [10]:
# Rounding variables based on use case
years_df['evapotranspiration_sum'] = years_df['evapotranspiration_sum'].round(4)
years_df['precipitation_sum'] = years_df['precipitation_sum'].round(2)
years_df['growing_degree_days'] = years_df['growing_degree_days'].round()
years_df['sunshine_avg'] = years_df['sunshine_avg'].round(2)
years_df['SPI'] = years_df['SPI'].round(2)

In [11]:
# Add wheat yield column on position 1 in main dataframe
years_df.insert(1, "Wheat_yield", df_wheat_yield['Yield'])

Normalizing wheat yield to account for technological progress

In [12]:
# Dictionary to store wheat yields
dict_df = years_df.set_index('Year')['Wheat_yield'].to_dict()


In [13]:
def wheat_yield_coefficient_WE(year):
    base_year = 1961
    transition_year_1 = 1980
    transition_year_2 = 2000
    end_year = 2019
    base_yield = 2.6  # Minimum yield from the table

    if year < base_year:
        return 0  # For years before 1961, no increase
    elif year <= transition_year_1:
        # 0.11 Tha−1 Year−1 increase from 1961 to 1980
        return 0.11 * (year - base_year)
    elif year <= transition_year_2:
        # 0.11 Tha−1 Year−1 increase from 1961 to 2000
        return 0.11 * (year - base_year)
    elif year <= end_year:
        # 0.11 Tha−1 Year−1 increase from 1961 to 2000, then 0.01 Tha−1 Year−1 increase from 2001 to 2019
        return 0.11 * (transition_year_2 - base_year) + 0.01 * (year - transition_year_2)
    else:
        # For years after 2019, continue with the 2001-2019 rate
        return 0.11 * (transition_year_2 - base_year) + 0.01 * (end_year - transition_year_2) + 0.01 * (year - end_year)

# Function to normalize yield, the data being in dt instead of tonnes.
def normalize_yield(year, value):
    return value - wheat_yield_coefficient_WE(year)*10

# To normalize the entire dataset:
normalized_yields = {year: normalize_yield(year, yield_value) for year, yield_value in dict_df.items()}

The function calculates the absolute increase in tonnes per hectare since 1961.
It uses the rates directly: 0.11 Tha−1 Year−1 from 1961 to 2000, and 0.01 Tha−1 Year−1 from 2001 onwards.
The wheat_yield_coefficient_WE function now returns the total increase in tonnes per hectare since 1961.
The normalize_yield function now subtracts the coefficient from the actual yield to get the 1961-equivalent yield.
We use the minimum yield (2.6 Tha−1) from the table(see documentation) as a base yield for 1961.

In [14]:
# Creating secondary dataframe with normalized yields
df_normalized = years_df.copy()

# Replace the yields with normalized yields
df_normalized['Wheat_yield'] = df_normalized['Year'].map(normalized_yields)

# Rounding yields
df_normalized['Wheat_yield'] = df_normalized['Wheat_yield'].round(1)
df_normalized.tail(10)

,Year,Wheat_yield,days_above_30degrees,precipitation_sum,sunshine_avg,growing_degree_days,evapotranspiration_sum,SPI
64,2014,42.0,3,346.23,10.40,2288.0,2458.8316,-0.51
65,2015,36.5,8,294.17,10.35,2166.0,2720.8699,-1.30
66,2016,31.9,1,382.98,9.88,2112.0,2533.9247,0.04
67,2017,31.8,1,464.92,9.51,2257.0,2936.4327,1.29
68,2018,22.0,14,280.18,10.50,2479.0,2798.7857,-1.51
69,2019,29.2,8,357.53,9.88,2326.0,3028.8592,-0.34
70,2020,33.3,6,315.53,10.41,2231.0,2884.6482,-0.98
71,2021,28.0,3,467.83,9.97,2063.0,2851.7016,1.33
72,2022,30.7,8,262.90,10.85,2296.0,2829.6136,-1.78
73,2023,29.1,4,459.93,9.79,2223.0,3084.4398,1.21


In [15]:
# Export the data
df_normalized.to_csv('wheat.csv', index=False)
print("DataFrame has been exported to csv")

DataFrame has been exported to csv
